# 基于SVM的文本分类实战
本教程告诉你如何使用SVM来实现一个基于词袋和支持向量机的简单文本分类。
## 载入数据
我们准备了中文的新闻数据作为样例数据集，其中训练数据条数50000，测试数据条数10000，所有数据分为体育、财经、房产、家居、教育、科技、时尚、时政、游戏和娱乐10类。从训练文本中，可以载入代码，查看数据格式和样例：

In [1]:
import codecs
import os
import jieba

In [21]:
train_file = '../resource/cnews/cnews.train.txt' # training data file name  
test_file = '../resource/cnews/cnews.test.txt'
vocab = '../resource/cnews/cnews.vocab.txt'

In [22]:
with codecs.open(train_file, 'r', 'utf-8') as f:
    lines = f.readlines()

# print sample content
label, content = lines[0].strip('\r\n').split('\t')
print(content)

马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有摆脱雨水的困扰。7月31日下午6点，国奥队的日常训练再度受到大雨的干扰，无奈之下队员们只慢跑了25分钟就草草收场。31日上午10点，国奥队在奥体中心外场训练的时候，天就是阴沉沉的，气象预报显示当天下午沈阳就有大雨，但幸好队伍上午的训练并没有受到任何干扰。下午6点，当球队抵达训练场时，大雨已经下了几个小时，而且丝毫没有停下来的意思。抱着试一试的态度，球队开始了当天下午的例行训练，25分钟过去了，天气没有任何转好的迹象，为了保护球员们，国奥队决定中止当天的训练，全队立即返回酒店。在雨中训练对足球队来说并不是什么稀罕事，但在奥运会即将开始之前，全队变得“娇贵”了。在沈阳最后一周的训练，国奥队首先要保证现有的球员不再出现意外的伤病情况以免影响正式比赛，因此这一阶段控制训练受伤、控制感冒等疾病的出现被队伍放在了相当重要的位置。而抵达沈阳之后，中后卫冯萧霆就一直没有训练，冯萧霆是7月27日在长春患上了感冒，因此也没有参加29日跟塞尔维亚的热身赛。队伍介绍说，冯萧霆并没有出现发烧症状，但为了安全起见，这两天还是让他静养休息，等感冒彻底好了之后再恢复训练。由于有了冯萧霆这个例子，因此国奥队对雨中训练就显得特别谨慎，主要是担心球员们受凉而引发感冒，造成非战斗减员。而女足队员马晓旭在热身赛中受伤导致无缘奥运的前科，也让在沈阳的国奥队现在格外警惕，“训练中不断嘱咐队员们要注意动作，我们可不能再出这样的事情了。”一位工作人员表示。从长春到沈阳，雨水一路伴随着国奥队，“也邪了，我们走到哪儿雨就下到哪儿，在长春几次训练都被大雨给搅和了，没想到来沈阳又碰到这种事情。”一位国奥球员也对雨水的“青睐”有些不解。


以训练数据的第一条为例，对载入的新闻数据进行分词，在这里我使用的LTP的切词功能，你也可以使用jieba，分词结果用"/"符号隔开展示。

In [23]:
# print word segment results
segment = jieba.cut(content)
print('/'.join(segment))

马晓旭/意外/受伤/让/国奥/警惕/ /无奈/大雨/格外/青睐/殷家/军/记者/傅亚雨/沈阳/报道/ /来到/沈阳/，/国奥队/依然/没有/摆脱/雨水/的/困扰/。/7/月/31/日/下午/6/点/，/国奥队/的/日常/训练/再度/受到/大雨/的/干扰/，/无奈/之下/队员/们/只/慢跑/了/25/分钟/就/草草收场/。/31/日/上午/10/点/，/国奥队/在/奥体中心/外场/训练/的/时候/，/天/就是/阴沉沉/的/，/气象预报/显示/当天/下午/沈阳/就/有/大雨/，/但/幸好/队伍/上午/的/训练/并/没有/受到/任何/干扰/。/下午/6/点/，/当/球队/抵达/训练场/时/，/大雨/已经/下/了/几个/小时/，/而且/丝毫/没有/停下来/的/意思/。/抱/着/试一试/的/态度/，/球队/开始/了/当天/下午/的/例行/训练/，/25/分钟/过去/了/，/天气/没有/任何/转好/的/迹象/，/为了/保护/球员/们/，/国奥队/决定/中止/当天/的/训练/，/全队/立即/返回/酒店/。/在/雨/中/训练/对/足球队/来说/并/不是/什么/稀罕/事/，/但/在/奥运会/即将/开始/之前/，/全队/变得/“/娇贵/”/了/。/在/沈阳/最后/一周/的/训练/，/国奥队/首先/要/保证/现有/的/球员/不再/出现意外/的/伤病/情况/以免/影响/正式/比赛/，/因此/这一/阶段/控制/训练/受伤/、/控制/感冒/等/疾病/的/出现/被/队伍/放在/了/相当/重要/的/位置/。/而/抵达/沈阳/之后/，/中/后卫/冯萧霆/就/一直/没有/训练/，/冯萧霆/是/7/月/27/日/在/长春/患上/了/感冒/，/因此/也/没有/参加/29/日/跟/塞尔维亚/的/热身赛/。/队伍/介绍/说/，/冯萧霆/并/没有/出现/发烧/症状/，/但/为了/安全/起/见/，/这/两天/还是/让/他/静养/休息/，/等/感冒/彻底/好/了/之后/再/恢复/训练/。/由于/有/了/冯萧霆/这个/例子/，/因此/国奥队/对雨中/训练/就/显得/特别/谨慎/，/主要/是/担心/球员/们/受凉/而/引发/感冒/，/造成/非战斗/减员/。/而/女足/队员/马晓旭/在/热身赛/中/受伤/导致/无缘/奥运/的/前科/，/也/让/在/沈阳/的/国奥队/现在/格外/警惕/，/“/训练/中/不断/嘱咐/队员/们

把上述逻辑稍微整理一下，实现一个类来载入训练和测试数据并进行分词。

In [24]:
# cut data
def process_line(idx, line):
    data = tuple(line.strip('\r\n').split('\t'))
    if not len(data)  == 2:
        return None
    content_segged = list(jieba.cut(data[1]))
    if idx % 1000 == 0:
        print('line number: {}'.format(idx))
    return (data[0], content_segged)

In [25]:
# data loading method
def load_data(file):
    with codecs.open(file, 'r', 'utf-8') as f:
        lines = f.readlines()
    data_records = [process_line(idx, line) for idx, line in enumerate(lines)]
    data_records = [data for data in data_records if data is not None]
    return data_records

In [27]:
# load and process training data
train_data = load_data(train_file)
print('first training data: label {} segment {}'.format(train_data[0][0], '/'.join(train_data[0][1])))

# load and process testing data
test_data = load_data(test_file)
print('first testing data: label {} segment {}'.format(test_data[0][0], '/'.join(test_data[0][1])))

line number: 0
line number: 1000
line number: 2000
line number: 3000
line number: 4000
line number: 5000
line number: 6000
line number: 7000
line number: 8000
line number: 9000
line number: 10000
line number: 11000
line number: 12000
line number: 13000
line number: 14000
line number: 15000
line number: 16000
line number: 17000
line number: 18000
line number: 19000
line number: 20000
line number: 21000
line number: 22000
line number: 23000
line number: 24000
line number: 25000
line number: 26000
line number: 27000
line number: 28000
line number: 29000
line number: 30000
line number: 31000
line number: 32000
line number: 33000
line number: 34000
line number: 35000
line number: 36000
line number: 37000
line number: 38000
line number: 39000
line number: 40000
line number: 41000
line number: 42000
line number: 43000
line number: 44000
line number: 45000
line number: 46000
line number: 47000
line number: 48000
line number: 49000
line number: 50000
first training data: label 体育 segment 马晓旭/意外

在花费一些时间进行分词后，可以开始构建词典了，词典从训练集中构建，按照词频进行排序。

In [28]:
def build_vocab(train_data, thresh):
    vocab = {'<UNK>': 0}
    word_count = {} # word frequency
    for idx, data in enumerate(train_data):
        content = data[1]
        for word in content:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
    word_list = [(k, v) for k, v in word_count.items()]
    print(f'word list length: {len(word_list)}')
    word_list.sort(key = lambda x : x[1], reverse = True) # sorted by word frequency
    word_list_filtered = [word for word in word_list if word[1] > thresh]
    print(f'word list length after filtering: {len(word_list_filtered)}')

    # construct vocab
    for word in word_list_filtered:
        vocab[word[0]]  = len(vocab)
    print(f'vocab size : {len(vocab)}')
    return vocab

In [29]:
vocab = build_vocab(train_data, 1)

word list length: 359255
word list length after filtering: 208966
vocab size : 208967


此外，根据category，我们知道标签本身也有个“词典”：

In [30]:
def build_label_vocab(cate_file):
    label_vocab = {}
    with codecs.open(cate_file, 'r', 'utf-8') as f:
        for lines in f:
            line = lines.strip().split('\t')
            label_vocab[line[0]]  = int(line[1])
    return label_vocab

In [31]:
label_vocab = build_label_vocab('../resource/cnews/cnews.category.txt')
print(f'label vocab: {label_vocab}')

label vocab: {'体育': 0, '娱乐': 1, '家居': 2, '房产': 3, '教育': 4, '时尚': 5, '时政': 6, '游戏': 7, '科技': 8, '财经': 9}


接下来构建id化的训练和测试集，因为我们只考虑bag of words，所以词的顺序被排除。构造成libsvm能吃的样子。注意，因为bag of word模型本 

In [32]:
def construct_trainable_matrix(corpus, vocab, label_vocab, out_file):
    records = []
    for idx, data in enumerate(corpus):
        if idx % 1000 == 0:
            print(f'process {idx} data')
        label = str(label_vocab[data[0]]) # label id
        token_dict = {}
        for token in data[1]:
            token_id = vocab.get(token, 0)
            if token_id in token_dict:
                token_dict[token_id] += 1
            else:
                token_dict[token_id] = 1
        feature = [str(int(k) + 1) + ':' + str(v) for k, v in token_dict.items()]
        feature_text = ' '.join(feature)
        records.append(label + ' ' + feature_text)

    with open(out_file, 'w') as f:
        f.write('\n'.join(records))

In [33]:
construct_trainable_matrix(train_data, vocab, label_vocab, '../resource/cnews/train.svm.txt')
construct_trainable_matrix(test_data, vocab, label_vocab, '../resource/cnews/test.svm.txt')

process 0 data
process 1000 data
process 2000 data
process 3000 data
process 4000 data
process 5000 data
process 6000 data
process 7000 data
process 8000 data
process 9000 data
process 10000 data
process 11000 data
process 12000 data
process 13000 data
process 14000 data
process 15000 data
process 16000 data
process 17000 data
process 18000 data
process 19000 data
process 20000 data
process 21000 data
process 22000 data
process 23000 data
process 24000 data
process 25000 data
process 26000 data
process 27000 data
process 28000 data
process 29000 data
process 30000 data
process 31000 data
process 32000 data
process 33000 data
process 34000 data
process 35000 data
process 36000 data
process 37000 data
process 38000 data
process 39000 data
process 40000 data
process 41000 data
process 42000 data
process 43000 data
process 44000 data
process 45000 data
process 46000 data
process 47000 data
process 48000 data
process 49000 data
process 0 data
process 1000 data
process 2000 data
process 3000

## 训练过程
剩下的核心模型反而简单：使用libsvm来训练支持向量机，让你的svm吃进你处理好的训练和测试文件，然后掉用libsvm的现有方法训练就行，我们可以更换不同的参数设定。libsvm的文档可以查看[这里](https://www.csie.ntu.edu.tw/~cjlin/libsvm/)，其中"-s,-t,-c"参数比较重要，他们决定你选择怎样的svm，你的核函数选择，你的惩罚系数。

In [34]:
from libsvm import svm
from libsvm.svmutil import svm_read_problem, svm_train, svm_predict, svm_save_model,svm_load_model

# train svm
train_label, train_feature = svm_read_problem('../resource/cnews/train.svm.txt')
print(train_label[0], train_feature[0])
model = svm_train(train_label, train_feature, '-s 0 -c 5 -t 0 -g 0.5 -e 0.1')

# predict
test_label, test_feature = svm_read_problem('../resource/cnews/test.svm.txt')
print(test_label[0], test_feature[0])
p_labs, p_acc, p_vals = svm_predict(test_label, test_feature, model)

0.0 {158685: 2.0, 1848: 1.0, 1562: 3.0, 56: 3.0, 12828: 2.0, 6198: 2.0, 9: 2.0, 2511: 2.0, 20324: 5.0, 4804: 2.0, 2645: 2.0, 1: 3.0, 4655: 1.0, 79: 1.0, 2457: 8.0, 273: 1.0, 1106: 1.0, 2: 38.0, 39883: 8.0, 670: 1.0, 54: 7.0, 4406: 1.0, 15446: 3.0, 3: 21.0, 3587: 1.0, 4: 14.0, 151: 2.0, 26: 2.0, 1162: 2.0, 47: 4.0, 1064: 4.0, 133: 2.0, 417: 3.0, 3648: 1.0, 1089: 13.0, 1328: 1.0, 592: 2.0, 4714: 2.0, 2332: 1.0, 2424: 3.0, 168: 4.0, 167: 1.0, 52111: 1.0, 7: 12.0, 676: 2.0, 661: 2.0, 27: 5.0, 65911: 1.0, 1595: 2.0, 103: 1.0, 5: 8.0, 46167: 1.0, 55879: 1.0, 154: 1.0, 871: 1.0, 84: 1.0, 104771: 1.0, 62933: 1.0, 247: 1.0, 1358: 3.0, 15: 2.0, 42: 3.0, 10139: 1.0, 2032: 3.0, 60: 3.0, 439: 2.0, 277: 1.0, 280: 2.0, 2765: 2.0, 15550: 1.0, 68: 1.0, 66: 1.0, 108: 2.0, 761: 1.0, 644: 1.0, 239: 1.0, 4656: 1.0, 15360: 1.0, 2976: 1.0, 3210: 1.0, 125: 2.0, 28511: 1.0, 1153: 1.0, 115: 2.0, 6990: 1.0, 451: 1.0, 2629: 1.0, 14677: 1.0, 2267: 1.0, 262: 2.0, 832: 1.0, 374: 4.0, 421: 1.0, 16494: 1.0, 2605: 2.0,

In [36]:
print(f'accuracy: {p_acc}')

accuracy: (94.43, 0.8064, 0.9065002155823192)


经过一段时间的训练，我们就可以观察到实验结果了，你可以更换不同的svm类型、惩罚系数和核函数，把结果调优。